In [1]:
import csv
from random import randint
from decimal import Decimal

import pandas as pd

from sqlalchemy.orm import Session
from sqlalchemy import update
from unicodedata import numeric


from models import Base, Customer, Account, Transaction
from db_sqlalchemy import init_db, engine
from etlfuncs import csv_reader_renamer, db_table_trasher, db_adder

In [ ]:
df = pd.read_csv('data/sebank_customers_with_accounts.csv')
df.head()
df['Municipality'] = df['Address'].str.split(' ').str[-1].str.strip()
df.head()
dict1 = df.to_dict('records')
print(dict1)

In [ ]:

# 
# for row in df.to_dict(orient='records'):
#     if row['Personnummer'] not in customers:
#         customers.append([row['Customer'], row['Address'], row['Phone'], row['Personnummer'], row['Municipality']])
# for row in df.to_dict(orient='records').values():
#     customers = {}
#     for col in row[:3]:
#         if col not in customers:
        

# 
# init_db()
# db = Session(engine)
# db.query(Customer).delete()
# db.query(Account).delete()

customers = []

for round, row in enumerate(dict1):
    ssn = row['Personnummer']
    name = row['Customer']
    address = row['Address']
    phone = row['Phone']
    account_number = row['BankAccount']
    municipality = row['Municipality']
    
    index = -1
    for i, customer in enumerate(customers):
        if customer["ssn"] == ssn:
            index = i
            break    

    if index == -1:
        new_customer = {
            "name": name, 
            "ssn": ssn, 
            "address": address, 
            "phone": phone, 
            "account_list": []
        } 
 
        new_customer["account_list"].append(account_number)     
        customers.append(new_customer)    
    else:        
        customers[index]["account_list"].append(account_number)  


In [ ]:



# random_balance = random.randint(500, 50000)
# float(random_balance)
# 
# 
# test_accounts = []
# 
# account1 = Account(account_number=1, municipality='One', customer_id=2)
# account2 = Account(account_number=2, municipality='Two', customer_id=3)
# account3 = Account(account_number=3, municipality='Three', customer_id=4)
# test_accounts.append(account3)
# test_accounts.append(account1)
# test_accounts.append(account2)
# 
# for account in test_accounts:
#     setattr(account, 'balance', float(random.randint(500, 10000)))
#     print(account.balance)




In [41]:
init_db()
db = Session(engine)
df = pd.read_csv('data/transactions.csv')
sample_df = df.sample(100)
pd.DataFrame.to_csv(sample_df, 'data/sample_transactions', index=False)
accounts = db.query(Account).all()


In [4]:
from itertools import count
exchange_rate_to_sek = {
    'SEK': 1.0,
    'DKK': 1.46,
    'USD': 9.68,
    'EUR': 10.89,
    'NOK': 0.94,
    'RMB': 1.34,
    'ZAR': 0.54,
    'GBP': 12.94,
    'ZMW': 0.36,
    'JPY': 0.067
}

init_db()
db = Session(engine)
accounts = db.query(Account).all()
account_numbers = []
for number in db.query(Account.account_number).all():
    account_numbers.append(number.account_number)    
    

with open('data/sample_transactions') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['transaction_type'] == 'outgoing':
            if not row['sender_account'] in account_numbers:
                if row['receiver_account'] in account_numbers:
                    update_account = db.query(Account).where(Account.account_number == row['receiver_account']).one_or_none()
                    old_balance = update_account.balance
                    if row['currency'] in exchange_rate_to_sek.keys():
                        converted_amount = Decimal(row['amount']) * Decimal(exchange_rate_to_sek[row['currency']])
                    update_account.balance = update_account.balance + converted_amount
                    
                    
                    
                    

23710.99
31782.52
30565.78
30362.35
8821.726599999999989640819820
13861.20419999999972979320972
27423.11
278669.0183999999920256840369
38899.05
46701.41
44136.11
52833.35519999999872098328524
329052.5359999999905404877154
323979.2847999999907280255229
13574.13
46442.48
17554.81
448534.6703999999869125758778
210793.2879999999939286681183
272763.8871999999921715840401
39488.91919999999898571729773
7983.843199999999995952748577
15085.03


In [81]:
for account in accounts:
    print(account.balance)

8446.0
7418.0
747.0
6046.0
7172.0
8522.0
1190.0
3465.0
5809.0
7034.0
3797.0
8111.0
3825.0
251.0
5937.0
1319.0
7536.0
4909.0
9811.0
1040.0
6374.0
7857.0
3237.0
8005.0
6104.0
6876.0
5669.0
622.0
4746.0
9555.0
5827.0
7250.0
6321.0
554.0
4119.0
5659.0
3971.0
8010.0
1234.0
6405.0
8891.0
6334.0
2640.0
4088.0
6976.0
9382.0
7533.0
7936.0
5478.0
2839.0
3717.0
8780.0
507.0
9045.0
5963.0
5354.0
5937.0
1229.0
9254.0
2601.0
7506.0
9093.0
7489.0
7059.0
4704.0
7432.0
369.0
6131.0
2024.0
5348.0
6708.0
6815.0
3037.0
4794.0
1854.0
2358.0
1793.0
6042.0
1838.0
614.0
9356.0
7537.0
5406.0
2929.0
3646.0
9199.0
1542.0
8502.0
3322.0
6319.0
5395.0
6709.0
7261.0
3329.0
7698.0
7242.0
5133.0
595.0
3761.0
3427.0
484.0
9607.0
6564.0
8565.0
8821.0
9742.0
7446.0
1547.0
3646.0
2707.0
8716.0
6423.0
9349.0
7946.0
4464.0
5688.0
3365.0
3807.0
994.0
5598.0
7861.0
7996.0
2083.0
1922.0
2292.0
2364.0
2165.0
713.0
4487.0
5827.0
951.0
8690.0
4156.0
5498.0
5791.0
8754.0
2673.0
9251.0
1272.0
6502.0
2957.0
7348.0
8579.0
1998.0
1743

[('SE8902EPWK73250364544965',),
 ('SE8902IDSK51225196610969',),
 ('SE8902OGIV86383792142837',),
 ('SE8902QZEZ52320024971424',),
 ('SE8902DWZI85436013187521',),
 ('SE8902UJGR36411837401781',),
 ('SE8902VAPD72818464374400',),
 ('SE8902VEIB72029945509184',),
 ('SE8902CHSI75448480838708',),
 ('SE8902XPHQ15953762308017',),
 ('SE8902VBHN50052224387585',),
 ('SE8902URSS59260219235990',),
 ('SE8902IQRO13056391101908',),
 ('SE8902GXLB05514229654020',),
 ('SE8902KWAF90270853352878',),
 ('SE8902MLNA56994057710583',),
 ('SE8902JIJC96071565969922',),
 ('SE8902BWXT39949824641151',),
 ('SE8902TKNC87869236455689',),
 ('SE8902PSRJ65756431691707',),
 ('SE8902SILE68578227700598',),
 ('SE8902UTKP18439002370878',),
 ('SE8902HQID54921038055598',),
 ('SE8902CMWE25531271390672',),
 ('SE8902ZWKL91581622045409',),
 ('SE8902NHEC01255630352421',),
 ('SE8902HRZI51889347255465',),
 ('SE8902AVQC54172602808950',),
 ('SE8902YHDH55973704252760',),
 ('SE8902BABM26306156053804',),
 ('SE8902UNAU13611272599268',),
 ('SE890

In [53]:
for account in account_numbers:
    print(account.account_number)
    
#GB88YSJL65468569134848

SE8902EPWK73250364544965
SE8902IDSK51225196610969
SE8902OGIV86383792142837
SE8902QZEZ52320024971424
SE8902DWZI85436013187521
SE8902UJGR36411837401781
SE8902VAPD72818464374400
SE8902VEIB72029945509184
SE8902CHSI75448480838708
SE8902XPHQ15953762308017
SE8902VBHN50052224387585
SE8902URSS59260219235990
SE8902IQRO13056391101908
SE8902GXLB05514229654020
SE8902KWAF90270853352878
SE8902MLNA56994057710583
SE8902JIJC96071565969922
SE8902BWXT39949824641151
SE8902TKNC87869236455689
SE8902PSRJ65756431691707
SE8902SILE68578227700598
SE8902UTKP18439002370878
SE8902HQID54921038055598
SE8902CMWE25531271390672
SE8902ZWKL91581622045409
SE8902NHEC01255630352421
SE8902HRZI51889347255465
SE8902AVQC54172602808950
SE8902YHDH55973704252760
SE8902BABM26306156053804
SE8902UNAU13611272599268
SE8902ZJVA11817265425330
SE8902JFQL02168511886094
SE8902COVG23577172278709
SE8902HWHC81157429988113
SE8902IDYW61560887680926
SE8902VBXW33852287376238
SE8902TTDD62882365317340
SE8902JDLY63782567946635
SE8902PKJC56529381306922


TypeError: 'DeclarativeMeta' object is not iterable